# Overview
This notebook is designed to perform number sorting tasks using machine learning and computer vision techniques. It utilizes the Dorna robot, a camera, and object detection algorithms to sort objects based on their numbers.

## Imports and parameters

In [ ]:
from dorna2 import Dorna
from camera import Camera
from dorna_vision import Detection
import time

In [ ]:
robot_ip_address = "192.168.1.100" # robot ip address
output_config = [0, 0, 1] # output in order (pin, off state, on state)
tcp = [0, 0, 0, 0, 0, 0] # tool length and position

place_positions = {
    1: [0, 0, 0, 0, 0, 0],  # Position for placing chip 1
    2: [0, 0, 0, 0, 0, 0],  # Position for placing chip 2
    3: [0, 0, 0, 0, 0, 0],  # Position for placing chip 3
    4: [0, 0, 0, 0, 0, 0],  # Position for placing chip 4
    5: [0, 0, 0, 0, 0, 0],  # Position for placing chip 5
}

middle_position = [0, 0, 0, 0, 0, 0]# here is the postion to robot will go to after picking but before placing
imaging_position = [1, 1, 1, 1, 1, 1] # here define the postion that the robot will take an image from when searching for objects
safe_joint = [1, 1, 1, 1 ,1 ,1 ]# here is the safe joint for robot to start
speed = 0.5 # here set the speed of the robot

# detection parameters
detection_parameters = {
    
}

## Initialization

In [ ]:
robot = Dorna() # initialize robot
robot.connect(robot_ip_address) # connect to robot

camera = Camera() # initialize camera
camera.connect() # connect to camera

od = Detection(robot, camera) # initialize the object detection

In [ ]:
robot.set_output(output_config[0], output_config[1]) # set output
robot.set_motor(1) # turn on the robot motors
robot.sleep(1) # sleep to allow the robot to settle before moving
robot.jmove(safe_joint)  #set safe initial position

## Main loop

In [ ]:
for i in range(5):  # range is set to 5 because we have 5 chips
    # run the object detection
    result = od.run(**detection_parameters)

    if result: # if there is an object detected

        pick_position = result[0]["tvec"] + [180, 0, 0] # get the pick position from the detection result
        object_class = result[0]["cls"] # get the class of the detected chip
        place_position = place_positions[object_class] # find the place position based on the chip's number

        # this executes the pick-and-place operation
        robot.pick_n_place(
            pick_pose=pick_position, # pick position we got from detection
            middle_pose=middle_position, # middle postion which we defined in the cfg cell
            place_pose=place_position, # place poistion which we got from the class
            end_pose=imaging_position, # end position which we will call the imaging postion because after ending  we want to scan again
            tcp=tcp, # tcp is the tool length and postion which we deffined in the cfg cell
            speed=speed, # speed which we also defined in the cfg cell
            output_config = output_config # output in order (pin, off state, on state)
        )

    time.sleep(1)

## Ending

In [ ]:
robot.close() # close robot
camera.close() # close camera
od.close() # close object detection